In [14]:
'''!curl --user dl-user https://static.miglix.eu/data/Task01_BrainTumour.tar | tar xf -'''

'!curl --user dl-user https://static.miglix.eu/data/Task01_BrainTumour.tar | tar xf -'

In [15]:
'''!curl -O --user dl-user https://static.miglix.eu/data/checkpoint_sd.pt'''

'!curl -O --user dl-user https://static.miglix.eu/data/checkpoint_sd.pt'

In [16]:
'''!rm -rf brats-net
!git clone https://github.com/hesiod/brats-net.git
!rm -f model
!ln -s brats-net/model .'''

'!rm -rf brats-net\n!git clone https://github.com/hesiod/brats-net.git\n!rm -f model\n!ln -s brats-net/model .'

In [2]:
import sys
!{sys.executable} -m pip install -r requirements.txt

The directory '/home/dave/.cache/pip/http' or its parent directory is not owned by the current user and the cache has been disabled. Please check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
The directory '/home/dave/.cache/pip' or its parent directory is not owned by the current user and caching wheels has been disabled. check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
    100% |████████████████████████████████| 753.2MB 2.8kB/s ta 0:00:0111 0% |▎                               | 7.3MB 8.1MB/s eta 0:01:32    4% |█▌                              | 35.7MB 7.9MB/s eta 0:01:32    7% |██▎                             | 53.1MB 7.0MB/s eta 0:01:41    7% |██▍                             | 55.0MB 7.0MB/s eta 0:01:40[K    30% |█████████▉                      | 232.6MB 120.1MB/s eta 0:00:05    31% |██████████                      | 236.7MB 8.1MB/s eta 0:01:05    49% |███████████████▉  

    100% |████████████████████████████████| 163kB 1.1MB/s ta 0:00:01
    100% |████████████████████████████████| 163kB 1.1MB/s ta 0:00:01
    100% |████████████████████████████████| 51kB 1.4MB/s ta 0:00:01
    100% |████████████████████████████████| 71kB 1.7MB/s ta 0:00:01
    100% |████████████████████████████████| 81kB 1.6MB/s ta 0:00:01
    100% |████████████████████████████████| 153kB 1.7MB/s ta 0:00:01
  Running setup.py install for grpcio ... done
  Running setup.py install for absl-py ... done


In [10]:
import torch
import torch.utils.data as td
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter
import os

In [13]:
import model.brats_dataset
import model.unet
import model.utils
import model.loss

In [17]:
class Context:
    def __init__(self, filename=None, criterion=None):
        self.net = model.unet.Net()
        if filename:
            self.net.load_state_dict(torch.load(filename))
        else:
            self.net.apply(model.unet.init_weights)

        self.device = model.utils.try_gpu()
        print('Using device {}'.format(self.device))

        self.run_iter = 0

    def check_topology(self):
        dummy_input = torch.randn(size=(2, 4, 240, 240), dtype=torch.float32)
        y = self.net(dummy_input)


    def export_onnx(self, filename):
        dummy_input = torch.randn(1, 4, 240, 240)
        torch.onnx.export(self.net, dummy_input, filename, verbose=True)

class TrainContext:
    def __init__(self, context, data_context, criterion, lr, batch_size, experiment_name='unnamed'):
        self.ctx = context
        self.data = data_context
        self.global_iter = 1
        self.experiment_name = experiment_name

        self.criterion = criterion
        self.batch_size = batch_size

        self.optimizer = torch.optim.SGD(self.ctx.net.parameters(), lr=lr)
        #optimizer = torch.optim.RMSprop(net.parameters(), lr=lr, weight_decay=1e-8, momentum=0.9)
        #self.scheduler = ReduceLROnPlateau(self.optimizer, 'min')

    def run(self, num_epochs):
        self.ctx.run_iter += 1
        run_name = 'runs/run_{}_{}_{}'.format(self.experiment_name, lr, self.ctx.run_iter)

        print('Commencing {}'.format(run_name))
        self.writer = SummaryWriter(os.path.join('runs', run_name))

        print('Writing graph')
        dummy_input = torch.randn(size=(2, 4, 240, 240), dtype=torch.float32)
        self.writer.add_graph(self.ctx.net, dummy_input)
        del dummy_input
        print('done')

        self.ctx.net.to(self.ctx.device)

        self.brats_train_perepoch = self.data.split_data(num_epochs)

        for epoch in range(num_epochs-1):
            self.run_epoch(epoch)

        self.writer.close()

    def checkpoint(self):
        torch.save(self.ctx.net.state_dict(), 'checkpoints/checkpoint_{}_{}.pt'.format(self.experiment_name, self.ctx.run_iter))

    def run_epoch(self, epoch):
        train_iter = td.DataLoader(self.brats_train_perepoch[epoch], batch_size, shuffle=True, num_workers=num_workers)
        test_iter = td.DataLoader(self.brats_train_perepoch[epoch+1], batch_size, shuffle=True, num_workers=num_workers)
        #test_iter = td.DataLoader(brats_test, batch_size, shuffle=False, num_workers=num_workers)
        
        batch_count = len(train_iter)
        
        print('Total batches: {}'.format(batch_count))

        train_loss_epoch = 0.0
        for i, (X, y) in enumerate(train_iter):
            batch_loss = self.run_batch(i, X, y)
            train_loss_epoch += batch_loss
            print('batch {:4}/{} batchloss {}'.format(i+1, batch_count, batch_loss))
        train_loss_epoch /= batch_count

        self.writer.add_scalar('loss/train', train_loss_epoch, self.global_iter)

        self.ctx.net.eval()
        test_loss_epoch = 0.0
        with torch.no_grad():
            for X_test, y_test in test_iter:
                X_test = X_test.float().to(self.ctx.device)
                y_test = y_test.float().to(self.ctx.device)
                y_test_hat = self.ctx.net(X_test).squeeze(1)
                b_l = self.criterion(y_test_hat, y_test)
                test_loss_epoch += float(b_l)
            test_loss_epoch /= len(test_iter)

        self.writer.add_scalar('loss/test', test_loss_epoch, self.global_iter)
        self.writer.flush()

        self.checkpoint()


        print('epoch {}/{}, train loss {}, test loss {}'.format(epoch+1,
            num_epochs, self.train_loss_epoch, test_loss_epoch))

    def run_batch(self, i, X, y):
        self.global_iter += 1
        self.ctx.net.train()

        X = X.float().to(self.ctx.device)
        y = y.float().to(self.ctx.device)
        y_hat = self.ctx.net(X).squeeze(1)

        l = self.criterion(y_hat, y)

        self.optimizer.zero_grad()
        l.backward()
        nn.utils.clip_grad_value_(self.ctx.net.parameters(), 0.1)
        self.optimizer.step()

        for tag, value in self.ctx.net.named_parameters():
            tag = tag.replace('.', '/')
            self.writer.add_histogram('weights/' + tag, value.data.cpu().numpy(), self.global_iter)
            self.writer.add_histogram('grads/' + tag, value.grad.data.cpu().numpy(), self.global_iter)

        self.writer.add_scalar('loss/total_loss', float(l), self.global_iter)
        self.writer.add_images('masks/0_base', X[:, 0:3, :, :], self.global_iter)
        y_us = y.unsqueeze(1)
        y_hat_us = y_hat.unsqueeze(1)
        y_hat_us_sig = torch.sigmoid(y_hat_us) > 0.5
        self.writer.add_images('masks/1_true', y_us, self.global_iter)
        self.writer.add_images('masks/2_predicted', y_hat_us_sig, self.global_iter)
        self.writer.add_images('extra/raw', y_hat_us, self.global_iter)
        overlaid = torch.cat([y_hat_us_sig.float(), y_us, torch.zeros_like(y_us)], dim=1)
        self.writer.add_images('extra/overlaid', overlaid, self.global_iter)

        self.writer.flush()

        return float(l)

In [18]:
dctx = model.brats_dataset.DataSplitter()

Loaded 484 files
Loaded 484 labels
Loaded 266 files


In [23]:
# Identifier for this group of runs
meta_name = 'brats4'
batch_size = 32
num_workers = 6
num_epochs = 50
lr = 0.0001
should_check = False

In [24]:
ctx = Context('checkpoint_sd.pt')

if should_check:
    # Check whether layer inputs/outputs dimensions are correct
    # by conducting a test run
    ctx.check_topology()

Using device cpu


In [26]:
pos_weight = torch.Tensor([5.0]).to(ctx.device)
criterion = model.loss.Loss(pos_weight)

tctx = TrainContext(ctx, dctx, criterion=criterion, lr=lr, batch_size=batch_size, experiment_name=meta_name)

In [27]:
tctx.run(num_epochs)

Commencing runs/run_brats4_0.0001_1
Writing graph
done
total count = 65340, num_epochs = 50, per epoch = 1306
Total batches: 41
batch    1/41 batchloss -0.41140273213386536
batch    2/41 batchloss -0.36365649104118347


KeyboardInterrupt: 

In [ ]:
ctx.export_onnx("net5.onnx")